# Process optimization by genetic algorithm

10 same items scheduling

In [125]:
include("../../src/ProcGA.jl")

Main.ProcGA

In [3]:
using CSV
using DelimitedFiles
using DataFrames
using Missings

In [59]:
using Plots, Plotly

In [60]:
plotly()

Plots.PlotlyBackend()

## reading data

In [4]:
ptbl = CSV.read("proctable.csv",header = 1);

In [5]:
# take smaller size 
ptbl = ptbl[1:10,:];

In [6]:
mtbl = convert(Array, ptbl[1:2])

10×2 Array{Any,2}:
 "BB105 間管"       50
 "BB201-3 3補助管2"  50
 "BH301M 間管"      50
 "BH301M 枝管1"     50
 "BH301M 枝管2"     50
 "EP201 間管"       50
 "EP201-2 2補助管"   50
 "EP201-2 枝管1"    50
 "HR567 枝管4"      50
 "HR567 枝管4C"     50

In [7]:
ptbl2 = [coalesce.(x,0) for x in convert(Array, ptbl[3:end])]

10×14 Array{Int64,2}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [8]:
s = string.(names(ptbl)[3:end])

14-element Array{String,1}:
 "pipecut"           
 "anneal_bs_prebend" 
 "anneal_ns_prebend" 
 "icing"             
 "rotbend"           
 "pressbend"         
 "anneal_bs_precalib"
 "anneal_ns_precalib"
 "vcalib"            
 "hcalib"            
 "buff_precut"       
 "mc_cut"            
 "buff_postcut"      
 "qc_check"          

In [9]:
nametable = DataFrame(id = 1:length(s), nm = s)

,id,nm
,Int64,String
1,1,pipecut
2,2,anneal_bs_prebend
3,3,anneal_ns_prebend
4,4,icing
5,5,rotbend
6,6,pressbend
7,7,anneal_bs_precalib
8,8,anneal_ns_precalib
9,9,vcalib


Read grouping info.

In [10]:
gtbl = readdlm("grouping.csv",',', skipstart = 1)

2×4 Array{Any,2}:
 "anneal_bs"  "[2;7]"  300  3
 "anneal_ns"  "[3;8]"  300  3

In [11]:
gtbl2=[]
for k in 1:size(gtbl)[1]
    x = gtbl[k, 2:end]
    push!(gtbl2,Dict(:id=>eval(Meta.parse(x[1])),:cnt=>x[2],:timespan=>x[3]))
end

In [12]:
gtbl2

2-element Array{Any,1}:
 Dict{Symbol,Any}(:timespan=>3,:id=>[2, 7],:cnt=>300)
 Dict{Symbol,Any}(:timespan=>3,:id=>[3, 8],:cnt=>300)

## Initialize ProcGA tables.

In [126]:
ProcGA.settable(ptbl2, gtbl2, mtbl)

()

## Initialize population

In [14]:
g1 = ProcGA.jobtablebase()

10×120 Array{Int64,2}:
 1  7  7  7  9  9  12  12  13  13  14  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0
 1  7  7  7  9  9  12  12  13  13  14     0  0  0  0  0  0  0  0  0  0  0  0

Read base data manually created from 6-same best practice.

In [17]:
bs = readdlm("10same.csv",',',Int)

10×35 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …   0   0   0   0   0  0  0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0      0  14  14   0   0  0  0   0   0
 0  0  1  0  7  7  7  9  9  12  12  13      0   0   0   0   0  0  0   0   0
 0  0  0  1  0  0  0  7  7   7   0   9      0   0   0   0   0  0  0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7     13   0   0  14  14  0  0   0   0
 0  0  0  0  0  1  0  7  7   7   0   0  …  14   0   0   0   0  0  0   0   0
 0  0  0  0  0  0  1  7  7   7   0   0      0   0   0   0   0  0  0   0   0
 0  0  0  0  0  0  0  1  0   0   7   7      0  13  13   0   0  0  0  14  14
 0  0  0  0  0  0  0  0  1   0   7   7      0   0   0   0   0  0  0   0   0
 0  0  0  0  0  0  0  0  0   1   0   0     13   0   0  14  14  0  0   0   0

In [18]:
ca = size(g1)[2]-size(bs)[2]

85

In [21]:
gbs = hcat(bs,zeros(Int,(10,ca)))

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [22]:
ProcGA.penalty(gbs)

35

In [23]:
ProcGA.checkvalidity(gbs)

false

In [24]:
ProcGA.validatejob!(gbs,10000)

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0  9  9  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [26]:
ProcGA.checkvalidity(gbs)

true

In [27]:
ProcGA.penalty(gbs)

102

Make population

In [28]:
ppl = ProcGA.initpopulationfrom(gbs,100);

In [98]:
vlst = []

0-element Array{Any,1}

## Evolution!

In [92]:
gbs0 = hcat(bs,zeros(Int,(10,ca)))

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [93]:
pushfirst!(ppl,gbs0);

In [129]:
# Proceed generation
vl = ProcGA.evolution!(ppl, 100, 10);

i:10 => (33, 33.0, 60)
i:20 => (33, 33.0, 43)
i:30 => (33, 33.0, 54)
i:40 => (33, 33.0, 49)
i:50 => (33, 33.0, 48)
i:60 => (33, 33.0, 51)
i:70 => (33, 33.0, 33)
i:80 => (33, 33.0, 33)
i:90 => (33, 33.0, 53)
i:100 => (33, 33.0, 43)


In [130]:
vlst = vcat(vlst,vl);

In [131]:
y = (x->x[1]).(vlst);
x = 1:length(y);

In [132]:
plot(x,y)

Pick up the best data.

In [133]:
v = ProcGA.penalty.(ppl)
idm = findmin(v)[2]
bestgen = ppl[idm]

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [134]:
ProcGA.penalty(bestgen)

33

In [135]:
ProcGA.clipjob(bestgen)

10×33 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …   0   0   0   0   0   0   0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0      0   0   0  14  14   0   0   0   0
 0  0  1  0  7  7  7  9  9  12  12  13      0   0   0   0   0   0   0   0   0
 0  0  0  1  0  0  0  7  7   7   0   9     14   0   0   0   0   0   0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7      0  13  13   0   0  14  14   0   0
 0  0  0  0  0  1  0  7  7   7   0   0  …  13  14  14   0   0   0   0   0   0
 0  0  0  0  0  0  1  7  7   7   0   0     14   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  1  0   0   7   7      0   0   0  13  13   0   0  14  14
 0  0  0  0  0  0  0  0  1   0   7   7      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0   1   0   0     12  13  13   0   0  14  14   0   0

In [136]:
ProcGA.validlength(bestgen)

33

Sort it by first appearance of process.

In [137]:
idx = zeros(Int,10)
for i in 1:10
    idx[i] = findfirst(x->(x>0),bestgen[i,:])
end
idx

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [138]:
id2 = sort(collect(1:10), by = x->idx[x])

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [139]:
bestgensort = similar(bestgen)
for i in 1:size(bestgen)[1]
    bestgensort[id2[i],:] = bestgen[i,:]
end
bestgensort

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [120]:
# save best process table to file.
# do not clip it so that it can be used again
writedlm("bestprc_20181112.csv",bestgensort, ',')

## read from saved data

In [127]:
rdlst = readdlm("bestprc_20181112.csv", ',', Int)

10×120 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  7  7  7  0  0   0   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  7  7  7  9  9  12  12  13     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  7  7   7   0   9     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  7  7   7   0   0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  7  7   7   0   0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1   0   7   7     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0   1   0   0     0  0  0  0  0  0  0  0  0  0  0  0

In [128]:
ppl = ProcGA.initpopulationfrom(rdlst,100);

## Conclusion

For given 10 same data with proctalbe below, the best combination is "bestgen" here.

Its length is 33.
Taking time unit as 1 hour and assuming job time as 7 hours/day, it needs almost 5 days to finish these materials.

In [140]:
transpose(ProcGA.proctable[1,:])

1×14 LinearAlgebra.Transpose{Int64,Array{Int64,1}}:
 1  0  0  0  0  0  3  0  2  0  0  2  2  2

In [141]:
ProcGA.clipjob(bestgen)

10×33 Array{Int64,2}:
 1  7  7  7  0  0  0  0  0   9   9   0  …   0   0   0   0   0   0   0   0   0
 0  1  0  0  7  7  7  0  0   0   0   0      0   0   0  14  14   0   0   0   0
 0  0  1  0  7  7  7  9  9  12  12  13      0   0   0   0   0   0   0   0   0
 0  0  0  1  0  0  0  7  7   7   0   9     14   0   0   0   0   0   0   0   0
 0  0  0  0  1  0  0  0  0   0   7   7      0  13  13   0   0  14  14   0   0
 0  0  0  0  0  1  0  7  7   7   0   0  …  13  14  14   0   0   0   0   0   0
 0  0  0  0  0  0  1  7  7   7   0   0     14   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  1  0   0   7   7      0   0   0  13  13   0   0  14  14
 0  0  0  0  0  0  0  0  1   0   7   7      0   0   0   0   0   0   0   0   0
 0  0  0  0  0  0  0  0  0   1   0   0     12  13  13   0   0  14  14   0   0